<a href="https://colab.research.google.com/github/matheusBernal/Detection-Faces/blob/main/DetectionFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install opencv-python opencv-python-headless opencv-contrib-python tensorflow matplotlib

In [21]:
from IPython import get_ipython
from IPython.display import display
import cv2
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import os
import cv2.data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import matplotlib.pyplot as plt
import pickle
import zipfile
from google.colab.patches import cv2_imshow

In [5]:
zipRef = zipfile.ZipFile("/content/data.zip", 'r')
zipRef.extractall()
zipRef.close()

In [29]:
train_dir = '/content/data/train'
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
for name in os.listdir(train_dir):
    path_class = train_dir+'/'+name
    for photo_name in os.listdir(path_class):
        path_name_class = path_class+'/'+photo_name
        image = cv2.imread(path_name_class)
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=4,minSize=(50,50))
        for i,(x,y,w,h) in enumerate(faces):
            Y = y+h
            X = x+w
            face = image[y:Y,x:X]
            face_resized = cv2.resize(face,(128,128))
            face_filename = f"/content/data/train_faces/{name}/{photo_name}"
            face_filename_valid = f"/content/data/valid/{name}/{photo_name}"
            cv2.imwrite(face_filename,face_resized)
            cv2.imwrite(face_filename_valid,face_resized)

In [45]:
TRAING_FACES_DIR = '/content/data/train_faces'
VALID_DIR = '/content/data/valid'

training_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [46]:
training_generator = training_datagen.flow_from_directory(TRAING_FACES_DIR,target_size=(300, 300),batch_size=30,class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(VALID_DIR,target_size=(300, 300),batch_size=30,class_mode='categorical')

Found 125 images belonging to 6 classes.
Found 125 images belonging to 6 classes.


In [44]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(300,300,3)),
    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(len(training_generator.class_indices),activation='softmax'),
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(training_generator,validation_data=validation_generator,epochs=26)

Epoch 1/26
5/5 ━━━━━━━━━━━━━━━━━━━━ 128s 25s/step - accuracy: 0.0860 - loss: 1.7960 - val_accuracy: 0.1680 - val_loss: 1.7911
Epoch 2/26
5/5 ━━━━━━━━━━━━━━━━━━━━ 142s 25s/step - accuracy: 0.1567 - loss: 1.7943 - val_accuracy: 0.1680 - val_loss: 1.7907
Epoch 3/26
5/5 ━━━━━━━━━━━━━━━━━━━━ 138s 25s/step - accuracy: 0.1441 - loss: 1.7911 - val_accuracy: 0.1680 - val_loss: 1.7892
Epoch 4/26
5/5 ━━━━━━━━━━━━━━━━━━━━ 121s 24s/step - accuracy: 0.1710 - loss: 1.7918 - val_accuracy: 0.1680 - val_loss: 1.7890
Epoch 5/26
5/5 ━━━━━━━━━━━━━━━━━━━━ 158s 28s/step - accuracy: 0.1671 - loss: 1.7879 - val_accuracy: 0.2800 - val_loss: 1.7874
Epoch 6/26
4/5 ━━━━━━━━━━━━━━━━━━━━ 16s 17s/step - accuracy: 0.2805 - loss: 1.7862

In [ ]:
accuracy = history.history['accuracy']
loss = history.history['loss']
val_accuracy = history.history['val_accuracy']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

In [ ]:
plt.plot(epochs,accuracy,'r',label='Training accuracy')
plt.plot(epochs,val_accuracy,'b',label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs,loss,'r',label='Training loss')
plt.plot(epochs,val_loss,'b',label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
from keras.preprocessing import image
def classificationImage(i):
    path_image = f"/content/data/test/{i}.jpg"
    image_loaded= image.load_img(path_image,target_size=(300,300))
    img_array = image.img_to_array(image_loaded)
    img_array = np.expand_dims(img_array,axis=0)
    img_array = np.vstack([img_array])
    predict = history.model.predict(img_array,batch_size=10)
    if predict[0][0]!=0:
        return "Amy"
    elif predict[0][1] !=0:
        return 'Bernadete'
    elif predict[0][2] !=0:
        return 'Leonard'
    elif predict[0][3] !=0:
        return 'Penny'
    elif predict[0][4] !=0:
        return 'Raj'
    else:
        return 'Sheldon'

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
image_cast = cv2.imread('/content/data/test/Cast.jpeg')
gray = cv2.cvtColor(image_cast,cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray,1.8,minNeighbors=4,minSize=(30,30))
for i,(x,y,w,h) in  enumerate(faces):
    #cv2.rectangle(image_cast,(x,y),(x+w,y+h),(255,0,0),2)
    Y = y+h
    X = x+w
    face = image_cast[y:Y,x:X]
    #cv2.imshow('Image cast',face)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    face_resized = cv2.resize(face,(300,300))
    face_filename = f"/content/data/test/{i}.jpg"
    cv2.imwrite(face_filename,face_resized)
    name_classification = classificationImage(i)
    cv2.rectangle(image_cast,(x,y),(x+w,y+h),(255,0,0),2)
    cv2.putText(image_cast,name_classification,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
cv2.imwrite('/content/output_image.jpg', image_cast)
#cv2.imshow(image_cast)